In [1]:
from notion_client import Client
import os 
import pandas as pd

NOTION = Client(auth=os.environ["NOTION_TOKEN"])
database_id = os.environ["NOTION_DATABASE_ID"]

In [2]:
def query_notion_database(database_id):
    results = []
    has_more = True
    start_cursor = None

    while has_more:
        response = NOTION.databases.query(
            database_id=database_id,
            start_cursor=start_cursor
        )
        results.extend(response["results"])
        has_more = response.get("has_more", False)
        start_cursor = response.get("next_cursor", None)

    return results

In [27]:
def extract_rows(results): 
    rows = []
    for page in results: 
        props = page["properties"]
        row = {
            "id": page["id"],
            "date": props.get("Date", {}).get("date", {}).get("start", None),
            "weight": props.get("Weight", {}).get("number", None),
            "time_of_day": props.get("Time of Day", {}).get("select", {}).get("name", None),
        }
        rows.append(row)

    return pd.DataFrame(rows)

In [74]:
weight_results = query_notion_database(database_id)
weight_df = extract_rows(weight_results)

In [ ]:
weight_results

In [ ]:
weight_df

In [ ]:
weight_df.columns

In [ ]:
import matplotlib.pyplot as plt

# Convert 'created_time' to datetime and extract the date
weight_df['date'] = pd.to_datetime(weight_df['date'])
# weight_df['day'] = weight_df['date'].dt.date

# Group by day and calculate the average weight
daily_avg_weight = weight_df.groupby('date')['weight'].mean()

# Plot the line chart
plt.figure(figsize=(10, 6))
plt.plot(daily_avg_weight.index, daily_avg_weight.values, marker='o', linestyle='-', color='b')
plt.ylim(70, 85)
plt.xlabel('date')
plt.ylabel('Weight (kg)')
plt.title('Average Weight by Day')
plt.grid(True)
plt.show()

### Load Notion data to Snowflake

In [ ]:
weight_df.head()

In [ ]:
weight_df['loaded_at'] = pd.to_datetime('now').isoformat()
weight_df['loaded_at'] = weight_df['loaded_at'].astype(str)
weight_df.head()

In [ ]:
weight_df['date']  = weight_df['date'].astype(str)
weight_df.dtypes

In [66]:
import snowflake.connector

conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA")
)

cursor = conn.cursor()

In [86]:
insert_query = """
INSERT INTO WEIGHT_DB.RAW.weight_logs_raw (id, date, weight_kg, time_of_day,loaded_at)
VALUES (%s, %s, %s, %s, %s)
"""

In [ ]:
weight_df.head()

In [ ]:
records = list(weight_df[["id", "date", "weight", "time_of_day", "loaded_at"]].itertuples(index=False, name=None))

cursor.executemany(insert_query, records)
conn.commit()

print(f"{cursor.rowcount} rows inserted.")


cursor.close()
conn.close()